- 네이버 영화 평점: https://movie.naver.com/movie/point/af/list.nhn
- 훈련 데이터: ratings_train.txt
- Test 데이터: ratings_test.txt
- 데이터: https://github.com/e9t/nsmc
    - 리뷰: document
    - 긍부정: label - 0:부정, 1:긍정

## 훈련/테스트 데이터 셋 읽기

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('../../1. Data/nsmc/ratings_train.txt', sep='\t', encoding='utf-8')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [3]:
test_df = pd.read_csv('../../1. Data/nsmc/ratings_test.txt', sep='\t', encoding='utf-8')
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [4]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

## 데이터셋 확인

In [11]:
train_df.isna().sum()

id          0
document    0
label       0
dtype: int64

In [10]:
test_df.isna().sum()

id          0
document    0
label       0
dtype: int64

In [8]:
train_df[train_df['document'].isna()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [9]:
# 결측치 처리 - 제거
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [12]:
#label(긍/부정 분포)
train_df['label'].value_counts()

0    75170
1    74825
Name: label, dtype: int64

In [13]:
test_df['label'].value_counts()

1    25171
0    24826
Name: label, dtype: int64

In [15]:
train_df.duplicated().sum()

0

## 텍스트 전처리

In [20]:
import string
import re
pattern = f'[{string.punctuation}]'
re.sub(pattern, ' ', "가나다.다라마!a#b^1*2$")
#print(pattern)

'가나다 다라마 a b 1 2 '

In [14]:
from konlpy.tag import Okt

In [29]:
# 특정 품사 (형용사, 부사, 조사..)만 조회

# N글자 이상인 token만 조회
def text_preprocessing(document):
    """document: 댓글하나"""
    okt = Okt()
    
    # 특수문자 제거
    pattern = f'[{string.punctuation}]'
    document = re.sub(pattern,' ', document)
    
    # 형태소 토큰화
    tokens = okt.morphs(document, norm=True)
    
    # 불용어 제거(추가)
    return ' '.join(tokens)

In [25]:
train_df.loc[0,'document']

'아 더빙.. 진짜 짜증나네요 목소리'

In [23]:
text_preprocessing(train_df.loc[0, 'document'])

'아 더빙 진짜 짜증나네요 목소리'

In [28]:
okt=Okt()
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [30]:
X_train = train_df['document'].apply(text_preprocessing)
X_test = test_df['document'].apply(text_preprocessing)

In [31]:
y_train = train_df['label']
y_test = test_df['label']

In [32]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((149995,), (149995,), (49997,), (49997,))

## Feature Vectorization

In [33]:
data = pd.concat([X_train, X_test])
data.shape

(199992,)

In [34]:
data.head()

0                                    아 더빙 진짜 짜증나네요 목소리
1                    흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나
2                            너 무재 밓었 다그 래서 보는것을 추천 한 다
3                         교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정
4    사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만 했...
Name: document, dtype: object

In [35]:
data.tail()

49995    오랜 만 에 평점 로 긴 했네 ㅋㅋ 킹왕짱 쌈뽕 한 영화 를 만났습니다 강렬하게 육 쾌함
49996     의지 박약 들 이나 하는거다 탈영 은 일단 주인공 김대희 닮았고 이등병 찐 따 OOOO
49997                    그림 도 좋고 완성 도도 높았지만 보는 내내 불안하게 만든다
49998    절대 봐 서는 안 될 영화 재미 도 없고 기분 만 잡 치고 한 세트 장 에서 다 해먹 네
49999                                         마무리 는 또 왜 이래
Name: document, dtype: object

In [36]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [52]:
# cv = CountVectorizer(min_df = 10)
cv = CountVectorizer(min_df = 10, ngram_range=(1,3))
cv.fit(data)
train_cv = cv.transform(X_train)
test_cv = cv.transform(X_test)

In [53]:
train_cv.shape, test_cv.shape

((149995, 27932), (49997, 27932))

In [54]:
# tfidf = TfidfVectorizer(min_df=10)
tfidf = TfidfVectorizer(min_df=10, ngram_range=(1,3))
tfidf.fit(data)
train_tfidf = tfidf.transform(X_train)
test_tfidf = tfidf.transform(X_test)

In [55]:
train_tfidf.shape, test_tfidf.shape

((149995, 27932), (49997, 27932))

In [42]:
train_cv[:3].toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [43]:
train_tfidf[:3].toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 긍부정 예측 머신러닝 모델링

In [56]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(max_iter=1000, random_state=1)
lg.fit(train_cv, y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [57]:
pred_train = lg.predict(train_cv)
pred_test = lg.predict(test_cv)

In [58]:
from sklearn.metrics import accuracy_score, classification_report


In [59]:
print('Train정확도 :', accuracy_score(y_train, pred_train))
print('Test정확도 :', accuracy_score(y_test, pred_test))
#print('class : ', )

Train정확도 : 0.8923897463248774
Test정확도 : 0.8381302878172691


In [60]:
lg2 = LogisticRegression(max_iter=1000, random_state=1)
lg2.fit(train_tfidf, y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [61]:
pred_train2 = lg2.predict(train_tfidf)
pred_test2 = lg2.predict(test_tfidf)

In [62]:
print('Train정확도 :', accuracy_score(y_train, pred_train2))
print('Test정확도 :', accuracy_score(y_test, pred_test2))

Train정확도 : 0.8751691723057435
Test정확도 : 0.842270536232174
